# Example notebook

This example is based on the aspirin test case. The code here is mostly adapted from the existing test infrastructure.

Here is the code copied from `<>/deepppl/deepppl/tests/inference`.

```
from .harness import MCMCTest
import numpy as np
from pprint import pprint

# Warning: Generated quantities does not work with numpyro on this example
# (In place mutation of array)

def test_aspirin():
    data = {}
    data['y'] = [2.77, 2.50, 1.84, 2.56, 2.31, -1.15]
    data['s'] = [1.65, 1.31, 2.34, 1.67, 1.98, 0.90]
    data['N'] = len(data['y'])
    data['mu_loc'] = np.mean(data['y'])
    data['mu_scale'] = 5 * np.std(data['y'])
    data['tau_scale'] = 2.5 * np.std(data['y'])
    data['tau_df'] = 4
     
    t_aspirin = MCMCTest(
        name='aspirin',
        model_file='deepppl/tests/good/aspirin.stan',
        data=data
    )

    return t_aspirin.run()
    
if __name__ == "__main__":
    pprint(test_aspirin())
```

The file `harness.py` has the data class that controls the test (`MCMCTest`), including the compile of stan, the execution through pyro and the comparison with the vanilla stan implementation.

Below is an example based on what `harness` implements.



## Setup the data

In [ ]:
import numpy as np

data = {}
data['y'] = [2.77, 2.50, 1.84, 2.56, 2.31, -1.15]
data['s'] = [1.65, 1.31, 2.34, 1.67, 1.98, 0.90]
data['N'] = len(data['y'])
data['mu_loc'] = np.mean(data['y'])
data['mu_scale'] = 5 * np.std(data['y'])
data['tau_scale'] = 2.5 * np.std(data['y'])
data['tau_df'] = 4

## Setup model

In [ ]:
# Configuration

from dataclasses import dataclass, field
import time
@dataclass
class Config:
    iterations: int = 1000
    warmups: int = 10
    chains: int = 4
    thin: int = 2
    file: str = "/opt/user/deepppl/deepppl/tests/good/aspirin.stan"
    
@dataclass
class TimeIt:
    name: str

    def __enter__(self):
        self.start = time.perf_counter()

    def __exit__(self, *exc_info):
        print(f"{self.name} took {time.perf_counter() - self.start}s to complete.")

import numpyro
import jax

# Request use the GPU
numpyro.set_platform("gpu")
print(f"jax version: {jax.__version__}")
print(f"numpyro version: {numpyro.__version__}")
print(f"jax target backend: {jax.config.FLAGS.jax_backend_target}")
print(f"jax target device: {jax.lib.xla_bridge.get_backend().platform}")

from deepppl import PyroModel, NumPyroModel

In [ ]:
with open(Config.file, 'r') as f:
    print(f.read())

# Stanc3 to numpyro test

In [ ]:
# Copy test file to exercise stanc3
!rm -rf /opt/user/stanc3-to-pyro-test
!mkdir /opt/user/stanc3-to-pyro-test
!cp -rf {Config.file} /opt/user/stanc3-to-pyro-test

# Add test directory to python path so that we can load the generated model afterwards.
import sys
sys.path.append('/opt/user/stanc3-to-pyro-test')

In [ ]:
# Compile file to numpyro and see result
!stanc --numpyro /opt/user/stanc3-to-pyro-test/aspirin.stan
!cat /opt/user/stanc3-to-pyro-test/aspirin.py

In [ ]:
# We could run the functions from aspirin right away. By importing it.
from aspirin import *


In [ ]:
# Well generate the model one more time, but thorugh the runtimes that come with stanc3.

In [ ]:
from os.path import join
from pathlib import Path
import json
from dataclasses import dataclass, field
from runtimes.pyro.dppl import PyroModel
from runtimes.numpyro.dppl import NumpyroModel

def testPyro(config, data):
    print(config.file)
    pyro_model = PyroModel(config.file, compile=True)
    mcmc = pyro_model.mcmc(config.iterations,
                           warmups=config.warmups,
                           chains=config.chains,
                           thin=config.thin)
    inputs = pyro_model.convert_inputs(data)
    mcmc.run(**inputs)
    return { 'samples': mcmc.get_samples() }

def testNumPyro(config, data):
    pyro_model = NumpyroModel(config.file, compile=True)
    mcmc = pyro_model.mcmc(config.iterations,
                           warmups=config.warmups,
                           chains=config.chains,
                           thin=config.thin)
    inputs = pyro_model.convert_inputs(data)
    mcmc.run(**inputs)
    return { 'samples': mcmc.get_samples() }


In [ ]:
# This didn't work for me on Power with libcu10.so error relatedwit initialisation of the CUDA devices.
# I suspect this is due to that library commming from some package compiled against a different version of CUDA

# samplesPyro = testPyro(Config, data)
# print(f'samplesPyro = {samplesPyro}')

In [ ]:
sampleNumPyro = testNumPyro(Config, data)
print(f'samplesPyro = {sampleNumPyro}')